
    This Source Code Form is subject to the terms of the Mozilla Public
    License, v. 2.0. If a copy of the MPL was not distributed with this
    file, You can obtain one at http://mozilla.org/MPL/2.0/.


Please be sure that all modules listed below are installed on your machine.

In [ ]:
import json, operator
import matplotlib.pyplot as plt
import matplotlib.transforms
import numpy as np
import pandas as pd
import seaborn as sns
from statsmodels.stats import weightstats as stests

### Global parameter to build seaborn boxplot

In [ ]:
# If necessary, update the path where all data are located.
# All computed p-values are gathered in a CSV file saved in the folder 'path'
# whose file name depends on the experiment results you want to see
path = './'
offset_path = len(path)

#beacs_no_bias = 'Debiased Beacs without CRACS without RA.json'
#beacs = 'Beacs without CRACS without RA.json'

beacs_no_bias = 'Debiased Beacs without CRACS with RA.json'
beacs = 'Beacs without CRACS with RA.json'

exp_list = [
    beacs_no_bias,
    beacs
]

exp_list_names = [
    'BEACS-debiased',
    'BEACS'
]

# Font size et figuresize parameters for plotting
figure_size = (20, 10)
axe_label_fontsize = '28'
tick_label_fontsize = '24'
legend_fontsize = '24'

# Significance level for p-values
alpha = 0.05
stat_file_name = 'stats'

# What to do ...
produce_graph = True
produce_stat = True

In [ ]:
def print_latex_line():
    with open(path + beacs_no_bias) as beacs_no_bias_json_file:
        with open(path + beacs) as beacs_json_file:
            beacs_no_bias_json = json.load(beacs_no_bias_json_file)
            beacs_no_bias_json.sort(key=operator.itemgetter('maze'))
            beacs_json = json.load(beacs_json_file)
            beacs_json.sort(key=operator.itemgetter('maze'))
            for beacs_no_bias_item, beacs_item in zip(beacs_no_bias_json, beacs_json):
                string_to_print = ""
                if beacs_no_bias_item['maze'][:-3] == "Woods101demi":
                    string_to_print += "Woods101.5"
                else:
                    string_to_print += beacs_no_bias_item['maze'][:-3]
                string_to_print += " & "
                string_to_print += str(round(beacs_item['avg_population'], 0))+"("+str(round(beacs_item['std_population'], 0))+")"
                string_to_print += " & "
                string_to_print += str(round(beacs_no_bias_item['avg_population'], 0))+"("+str(round(beacs_no_bias_item['std_population'], 0))+")"
                string_to_print += " & "
                string_to_print += str(round(beacs_item['avg_knowledge'], 2))+"("+str(round(beacs_item['std_knowledge'], 2))+")"
                string_to_print += " & "
                string_to_print += str(round(beacs_no_bias_item['avg_knowledge'], 2))+"("+str(round(beacs_no_bias_item['std_knowledge'], 2))+")"
                string_to_print += " & "
                string_to_print += str(round(beacs_item['avg_ep_error_list'], 2))+"("+str(round(beacs_item['std_ep_error_list'], 2))+")"
                string_to_print += " & "
                string_to_print += str(round(beacs_no_bias_item['avg_ep_error_list'], 2))+"("+str(round(beacs_no_bias_item['std_ep_error_list'], 2))+")"
                string_to_print += " & "
                string_to_print += str(round(beacs_item['avg_exploit_rl'], 2))+"("+str(round(beacs_item['std_exploit_rl'], 2))+")"
                string_to_print += " & "
                string_to_print += str(round(beacs_no_bias_item['avg_exploit_rl'], 2))+"("+str(round(beacs_no_bias_item['std_exploit_rl'], 2))+") \\\\"
                print(string_to_print)

### Please, do not modify any lines of code below

### Reading json files

In [ ]:
def read_json(path, exp_list):
    raw_data = []
    for name in exp_list:
        json_data_from_lcs = path + name
        with open(json_data_from_lcs) as json_file:
            raw_data_from_lcs = json.load(json_file)
            raw_data.append(raw_data_from_lcs)
    return raw_data

# Call function to prepare plotting data
raw_data = read_json(path, exp_list)

### Preparing pandas plotting data

In [ ]:
def prepare_pandas_plotting_data(raw_data, exp_list, exp_list_names):
    cleaned_data = {
        'LCS':[],
        'Maze':[],
        'Knowledge (%)':[],
        'Population of classifiers':[],
        'Reliable classifiers ratio':[],
        'Mean realiable classifier specificity':[],
        'Average EP Accumulated Error (%)':[],
        'Average steps to exit':[]
    }
    nb_of_environments = 0
    for i in range(len(exp_list_names)):
        for item in raw_data[i]:
            if 'time' not in item.keys():
                nb_of_environments += 1
                for idx in range(30):
                    cleaned_data['LCS'].append(exp_list_names[i])
                    if item['maze'][:-3] == "Woods101demi":
                        cleaned_data['Maze'].append("Woods101.5")
                    else:
                        cleaned_data['Maze'].append(item['maze'][:-3])
                    cleaned_data['Knowledge (%)'].append(
                        item['knowledge_list'][idx]
                    )
                    cleaned_data['Population of classifiers'].append(
                        item['population_list'][idx]
                    )
                    cleaned_data['Reliable classifiers ratio'].append(
                        float(item['reliable_list'][idx]) / item['population_list'][idx]
                    )
                    cleaned_data['Mean realiable classifier specificity'].append(
                        item['mean_reliable_classifier_specificity_list'][idx]
                    )
                    cleaned_data['Average EP Accumulated Error (%)'].append(
                        item['ep_error_list'][idx]
                    )
                    cleaned_data['Average steps to exit'].append(
                        item['avg_exploit_rl_list'][idx]
                    )

    nb_of_environments /= len(exp_list)
    pandas_data = pd.DataFrame(cleaned_data)
    sorted_pandas_data = pandas_data.sort_values("Maze")
    return nb_of_environments, sorted_pandas_data

nb_of_environments, pandas_data = prepare_pandas_plotting_data(raw_data, exp_list, exp_list_names)

### Plotting boxplots with seaborn

In [ ]:
rc={
    'axes.labelsize':axe_label_fontsize,
    'xtick.labelsize':tick_label_fontsize,
    'ytick.labelsize':tick_label_fontsize
}
sns.set(context='notebook', style='ticks', rc=rc)

In [ ]:
def plot_figure(pandas_data, Y, lcs_name):
    
    #palette = sns.color_palette('pastel')
    #color1=palette[0]
    #color2=palette[2]
    #pal = [color1, color2]
    
    # Build the main figure
    fig, ax = plt.subplots(figsize=figure_size)
    sns.stripplot(
        x = 'Maze',
        y = Y,
        hue = 'LCS',
        data = pandas_data,
        edgecolor='gray',
        jitter=True,
        linewidth = 1,
        dodge=True,
        #palette=pal,
        palette='pastel',
        ax = ax
    )
    sns.boxplot(
        x = 'Maze',
        y = Y,
        hue = 'LCS',
        data = pandas_data,
        fliersize=0,
        #palette=pal,
        palette='pastel',
        ax = ax
    )

    # Set up x tick labels and x label correctly
    xlocs, xlabels = plt.xticks() 
    plt.xticks(np.arange(nb_of_environments)+0.5, xlabels, rotation = 45, horizontalalignment = 'right')
    plt.xlabel('')

    # Hide the horizontal gridlines
    ax.yaxis.grid(False)

    # Show the vertical gridlines
    ax.xaxis.grid(True)

    # Set up minor ticks and align label of major ticks on minor ticks
    ax.xaxis.set_minor_locator(plt.MultipleLocator(0.5))
    dx = (((figure_size[0] * 100 / 4.))/nb_of_environments - 10)/72.; dy = 0/72. # Width in inches converted into pixels depending on default DPI
    offset = matplotlib.transforms.ScaledTranslation(dx, dy, fig.dpi_scale_trans)
    for label in ax.xaxis.get_majorticklabels():
        label.set_transform(label.get_transform() - offset)

    # Set up legend
    handles, _ = ax.get_legend_handles_labels()
    plt.legend(handles[0:len(lcs_name)], lcs_name[0:len(lcs_name)], frameon=False, fontsize=legend_fontsize, bbox_to_anchor=(0.5, 1.05), borderaxespad=0., loc='center', ncol=3)


In [ ]:
def plot_whole_population_and_reliable_ratio(pandas_data, lcs_name):
    # Build the main figure
    fig, ax = plt.subplots(1, 2, figsize=figure_size)
    
    sns.stripplot(
        x = 'Maze',
        y = 'Population of classifiers',
        hue = 'LCS',
        data = pandas_data,
        edgecolor='gray',
        jitter=True,
        linewidth = 1,
        dodge=True,
        #palette=pal,
        palette='pastel',
        ax = ax[0]
    )
    sns.boxplot(
        x = 'Maze',
        y = 'Population of classifiers',
        hue = 'LCS',
        data = pandas_data,
        fliersize=0,
        #palette=pal,
        palette='pastel',
        ax = ax[0]
    )
    
    handles, labels = ax[0].get_legend_handles_labels()
    fig.legend(handles[0:len(lcs_name)], lcs_name[0:len(lcs_name)], frameon=False, fontsize=legend_fontsize, borderaxespad=1., loc='upper center', ncol=3)
    
    # Get first subplot correctly
    plt.sca(ax[0])
    
    # Set up x tick labels and x label correctly
    xlocs, xlabels = plt.xticks() 
    plt.xticks(np.arange(nb_of_environments)+0.5, xlabels, rotation = 45, horizontalalignment = 'right', fontsize=16)
    plt.xlabel('')

    # Hide the horizontal gridlines
    ax[0].yaxis.grid(False)

    # Show the vertical gridlines
    ax[0].xaxis.grid(True)

    # Set up minor ticks and align label of major ticks on minor ticks
    ax[0].xaxis.set_minor_locator(plt.MultipleLocator(0.5))
    #dx = (((figure_size[0] * 100 / 4.))/nb_of_environments - 10)/72.; dy = 0/72. # Width in inches converted into pixels depending on default DPI
    #offset = matplotlib.transforms.ScaledTranslation(dx, dy, fig.dpi_scale_trans)
    #for label in ax[0].xaxis.get_majorticklabels():
    #    label.set_transform(label.get_transform() - offset)
        
    # Set up legend
    ax[0].get_legend().remove()

    sns.stripplot(
        x = 'Maze',
        y = 'Reliable classifiers ratio',
        hue = 'LCS',
        data = pandas_data,
        edgecolor='gray',
        jitter=True,
        linewidth = 1,
        dodge=True,
        palette='pastel',
        ax = ax[1]
    )
    sns.boxplot(
        x = 'Maze',
        y = 'Reliable classifiers ratio',
        hue = 'LCS',
        data = pandas_data,
        fliersize=0,
        palette='pastel',
        ax = ax[1]
    )
    
    # Get second subplot correctly
    plt.sca(ax[1])
    
    # Set up x tick labels and x label correctly
    xlocs, xlabels = plt.xticks() 
    plt.xticks(np.arange(nb_of_environments)+0.5, xlabels, rotation = 45, horizontalalignment = 'right', fontsize=16)
    plt.xlabel('')

    # Hide the horizontal gridlines
    ax[1].yaxis.grid(False)

    # Show the vertical gridlines
    ax[1].xaxis.grid(True)

    # Set up minor ticks and align label of major ticks on minor ticks
    ax[1].xaxis.set_minor_locator(plt.MultipleLocator(0.5))
    #dx = (((figure_size[0] * 100 / 4.))/nb_of_environments - 10)/72.; dy = 0/72. # Width in inches converted into pixels depending on default DPI
    #offset = matplotlib.transforms.ScaledTranslation(dx, dy, fig.dpi_scale_trans)
    #for label in ax[1].xaxis.get_majorticklabels():
    #    label.set_transform(label.get_transform() - offset )

    # Set up legend
    ax[1].get_legend().remove()

In [ ]:
def plot_knowledge(pandas_data, zipped:bool=True):
    plot_figure(pandas_data, Y='Knowledge (%)', lcs_name=exp_list_names)

def plot_mean_classifier_specificity(pandas_data):
    plot_figure(pandas_data, Y='Mean realiable classifier specificity', lcs_name=exp_list_names)

def plot_ep_accumulated_error(pandas_data):
    plot_figure(pandas_data, Y='Average EP Accumulated Error (%)', lcs_name=exp_list_names)

def plot_avg_steps_to_exit(pandas_data):
    plot_figure(pandas_data, Y='Average steps to exit', lcs_name=exp_list_names)


In [ ]:
if produce_graph:
    plot_knowledge(pandas_data)

In [ ]:
if produce_graph:
    plot_whole_population_and_reliable_ratio(pandas_data, exp_list_names)

In [ ]:
if produce_graph:
    plot_mean_classifier_specificity(pandas_data)

In [ ]:
if produce_graph:
    plot_ep_accumulated_error(pandas_data)

In [ ]:
if produce_graph:
    plot_avg_steps_to_exit(pandas_data)

### Preparing statistical data from raw read json

In [ ]:
if produce_stat:
    raw_list_by_env_name = {}
    for i in range(len(exp_list_names)):
        for item in raw_data[i]:
            if 'time' not in item.keys():
                if item['maze'] not in raw_list_by_env_name.keys():
                    raw_list_by_env_name[item['maze']] = {
                        'knowledge_list' : [],
                        'population_list' : [],
                        #'reliable_ratio_list' : [],
                        #'mean_reliable_classifier_specificity_list' : [],
                        'EP_Accumulated_Error' : [],
                        'Average_steps_to_exit' : []
                    }
                raw_list_by_env_name[item['maze']]['knowledge_list'].append(
                    np.array(item['knowledge_list']))
                raw_list_by_env_name[item['maze']]['population_list'].append(
                    np.array(item['population_list']))
                #raw_list_by_env_name[item['maze']]['reliable_ratio_list'].append(
                #    np.array( 
                #        [float(b) / m for b,m in zip(item['reliable_list'], item['population_list'])]
                #    ))
                #raw_list_by_env_name[item['maze']]['mean_reliable_classifier_specificity_list'].append(
                #    np.array(item['mean_reliable_classifier_specificity_list']))
                raw_list_by_env_name[item['maze']]['EP_Accumulated_Error'].append(
                    np.array(item['ep_error_list']))
                raw_list_by_env_name[item['maze']]['Average_steps_to_exit'].append(
                    np.array(item['avg_exploit_rl_list']))

### Computing p-values

In [ ]:
if produce_stat:
    alcs = ""+exp_list_names[0]+"-"+exp_list_names[1]+""
    raw_statistical_data = {
        'Maze':[],
        'Metric':[],
        'ALCS':[],
        'Alternative':[],
        'Degrees of freedom':[],
        'P Value':[],
        'Null Hypothesis':[]
    }
    for env_name in raw_list_by_env_name.keys():
        for metric in raw_list_by_env_name[env_name].keys():
            for i in range(1):
                raw_statistical_data['Maze'].append(env_name)
                raw_statistical_data['Metric'].append(metric)
                raw_statistical_data['ALCS'].append(alcs)
                x1 = np.array(raw_list_by_env_name[env_name][metric][0])
                x2 = np.array(raw_list_by_env_name[env_name][metric][1])
                if i == 0:
                    alt = 'two-sided'
                elif i == 1:
                    alt = 'larger'
                else:
                    alt = 'smaller'
                raw_statistical_data['Alternative'].append(alt)
                tstats, pvalue, dof = stests.ttest_ind(x1, x2, alternative=alt, usevar='unequal', value=0)
                raw_statistical_data['Degrees of freedom'].append(dof)
                raw_statistical_data['P Value'].append(pvalue)
                if pvalue<alpha:
                    raw_statistical_data['Null Hypothesis'].append('Reject')
                else:
                    raw_statistical_data['Null Hypothesis'].append('Accept')
    statistical_pandas = pd.DataFrame(raw_statistical_data)

In [ ]:
if produce_stat:
    statistical_pandas.to_csv(path + stat_file_name + '.csv')